In [13]:
# ruff: noqa: E402
import math
import warnings
from typing import Dict, Literal
import time

warnings.simplefilter("ignore")
import delu  # Deep Learning Utilities: https://github.com/Yura52/delu
import numpy as np
import scipy.special
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn.functional as F
import torch.optim
from torch import Tensor
from tqdm.std import tqdm
import json
import sys

warnings.resetwarnings()

from rtdl_revisiting_models import MLP, ResNet, FTTransformer

from fourierDistill import *
from featurizer import BinaryTransformer

In [2]:
from mdlp.discretization import MDLP

/home/mattyshen/anaconda3/lib/python3.9/site-packages/mdlp/discretization.py:107: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  min_split=1e-3, dtype=np.int):


In [3]:
#dataset = sklearn.datasets.fetch_california_housing(as_frame = True)
dataset = sklearn.datasets.load_iris(as_frame = True)
X: np.ndarray = dataset["data"]
Y: np.ndarray = dataset["target"]

all_idx = np.arange(len(Y))
train_idx, test_idx = sklearn.model_selection.train_test_split(
    all_idx, train_size=0.8, random_state = 0
)
# train_idx, val_idx = sklearn.model_selection.train_test_split(
#     trainval_idx, train_size=0.8, random_state = 0
# )
X_train = X.loc[train_idx]
X_test = X.loc[test_idx]
y_train = Y.loc[train_idx]
y_test = Y.loc[test_idx]

In [4]:
n_inter = 3
k_cv = 3

In [5]:
bt_bin3 = BinaryTransformer(depth = 3, bit = True)
X_train_bin3 = bt_bin3.fit_and_transform(X.loc[train_idx, :], Y.loc[train_idx])
X_test_bin3 = bt_bin3.transform(X.loc[test_idx, :])

In [6]:
X_train_bin3.shape

(120, 12)

In [11]:
ftd_bin3 = FTDistillClassifierCV(
                 pre_interaction=None, 
                 pre_lam1=0.01, 
                 pre_lam2=0.01,
                 pre_max_features=None,
                 post_interaction='l1l2', 
                 post_lam1=0.1, 
                 post_lam2=0.1,
                 post_max_features=None,
                 size_interactions=3)


start = time.time()
ftd_bin3.fit(X_train_bin3, y_train, bt_bin3.no_interaction)
end = time.time()

Post-interaction model fitting
(120, 299)
Re-fitting with LogisticRegression with L1 penalty


In [8]:
ftd_bin3.re_fit_alpha

10.0

In [9]:
ftd_bin3.post_interaction_model.C

AttributeError: 'ElasticNetCV' object has no attribute 'C'

In [ ]:
ftd_bin3.post_interaction_model.C

In [12]:
np.mean(ftd_bin3.predict(X_test_bin3)== y_test)

0.9666666666666667

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(ftd_bin3.predict(X_train_bin3), y_train)

In [ ]:
r2_score(ftd_bin3.predict(X_test_bin3), y_test)

In [ ]:
m = MDLP()

In [ ]:
X_train_d = m.fit_transform(X_train, y_train)

In [ ]:
X_test_d = m.transform(X_test)

In [ ]:
X_test_d

In [ ]:
ftd_bin3.post_lam2

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
model_list = [ftd_bin3, ftd_bit3, ftd_bit4]
model_names = ['(bin3, true, train)', '(bit3, true, train)', '(bit4, true, train)']

r2_df = pd.DataFrame()
r2_df['Model'] = model_names

In [ ]:
r2_df['Train R2'] = [r2_score(ftd_bin3.predict(X_train_bin3), y_train),r2_score(ftd_bit3.predict(X_train_bit3), y_train), r2_score(ftd_bit4.predict(X_train_bit4), y_train)]
r2_df['Test R2'] = [r2_score(ftd_bin3.predict(X_test_bin3), y_test),r2_score(ftd_bit3.predict(X_test_bit3), y_test), r2_score(ftd_bit4.predict(X_test_bit4), y_test)]

In [ ]:
r2_df['Train Time'] = train_time
r2_df['Total Num Features'] = [len(m.regression_model.coef_) for m in model_list]
r2_df['Num Selected Features'] = [sum(m.regression_model.coef_ != 0) for m in model_list]

In [ ]:
r2_df

In [ ]:
r2_df.to_csv('r2/binarize_prediction.csv')

In [ ]:
[m.regression_model.reg_param for m in model_list]

In [ ]:
ftd_bit4.regression_model.alpha_

In [ ]:
# Import necessary libraries
import numpy as np
import l0learn

# Generate synthetic data
np.random.seed(0)
n_samples, n_features = 100, 20
X = np.random.randn(n_samples, n_features)
true_coef = np.zeros(n_features)
true_coef[::2] = 1  # Only every other feature is relevant
y = np.dot(X, true_coef) + np.random.normal(size=n_samples)

# Fit the model using l0learn
fit = l0learn.fit(X, y, max_support_size=10)

# Print the fitted coefficients for the best model
print("Fitted coefficients:")
#print(fit.coef_)

# # Make predictions
# X_new = np.random.randn(10, n_features)  # New data for prediction
# predictions = l0learn_predict(fit, X_new)
# print("Predictions:")
# print(predictions)


In [ ]:
fit

In [ ]:
fit.coeff(lambda_0=0.079901, gamma=0).toarray()

In [ ]:
fit.coeff(lambda_0 = 0.079901, gamma = 0)

In [ ]:
import numpy as np
np.random.seed(4) # fix the seed to get a reproducible result
n, p, k = 500, 1000, 10
X = np.random.normal(size=(n, p))
B = np.zeros(p)
B[:k] = 1
e = np.random.normal(size=(n,))/2
y = X@B + e

In [ ]:
fit_model = l0learn.fit(X, y, penalty="L0", max_support_size=20)

In [ ]:
fit_model

In [ ]:
fit_model_2 = l0learn.fit(X, y, penalty="L0L2", num_gamma = 5, gamma_min = 0.0001, gamma_max = 10, max_support_size=20)

In [ ]:
fit_model_2.coeff(lambda_0=0.0016, gamma=10)

In [ ]:
import numpy as np
from l0bnb import fit_path
from l0bnb import gen_synthetic

"""
For demonstration, we first generate a synthetic regression dataset (X,y)
as follows: y = X*b + epsilon, where the true vector of coefficients b
is sparse and has only 10 nonzero entries.
We set the number of samples n=1000 and number of features p=10,000.
"""
X, y, b = gen_synthetic(n=1000, p=10000, supp_size=10)
print("Nonzero indices in b: ", np.nonzero(b)[0])

"""
Run L0BnB to solve the problem for a sequence of lambda_0's.
By default, the sequence of lambda_0's is automatically chosen by the toolkit.
Use max_nonzeros=10 to stop the regularization path when it exceeds 10 nonzeros.
Here we fix lambda_2 = 0.01 (generally, this is data-dependent).
"""
sols = fit_path(X, y, lambda_2 = 0.01, max_nonzeros = 10)

"""
sols is a list of solutions, each corresponding to a different lambda_0.
Below we inspect the solution with index 4.
The estimated coefficients vector "b_estimated" and the intercept term can be accessed as follows:
"""
b_estimated = sols[4]["B"] # a numpy array.
intercept = sols[4]["B0"]

# To check the nonzero indices in b_estimated:
print("Nonzero indices in b_estimated: ", np.nonzero(b_estimated)[0])
# The nonzero indices in b_estimated match that of b.

# Predictions on the training data can be made as follows:
y_estimated = np.dot(X, b_estimated) + intercept

# For more advanced usage, check the documentation of fit_path:
print(fit_path.__doc__)


In [ ]:
import numpy as np
from irf import irf_utils
from irf.ensemble import RandomForestClassifierWithWeights
